<a href="https://colab.research.google.com/github/fwetfgwe4gr/efed-baedon/blob/main/DeepSaudiWildLifeCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Step 1: Import necessary libraries and load the dataset

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import matplotlib.pyplot as plt
import random

In [ ]:
 Load the dataset by iterating through the folders and creating a list of image paths and their corresponding labels:

In [4]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Set the path to your dataset folder (inside Google Drive)
dataset_dir = '/content/gdrive/MyDrive/Sample'

# Create lists to store image paths and labels
image_paths = []
labels = []

# Iterate through the folders
for folder in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder)
    if os.path.isdir(folder_path):
        # Get the label from the folder name
        label = folder

        # Iterate through the images in the folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if file.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                image_paths.append(file_path)
                labels.append(label)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Choose a class to display images from
# Note: This is for testing purposes only to find initially if data is correctly loaded
class_index = 55  # Choose a class index (0-143)

# Get the label of the chosen class
class_label = list(set(labels))[class_index]

# Get the image paths for the chosen class
class_image_paths = [img_path for img_path, label in zip(image_paths, labels) if label == class_label]

# Display the first 9 images from the chosen class
plt.figure(figsize=(10, 10))
for i, img_path in enumerate(class_image_paths[:9]):
    img = load_img(img_path, target_size=(224, 224))
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(class_label)
    plt.axis('off')
plt.show()

In [ ]:
Step 2: Data augmentation and preprocessing

Create an ImageDataGenerator to apply data augmentation and preprocessing to the images:

In [98]:
# Define the data augmentation and preprocessing parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Create a data generator for training
train_datagen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=8,  # Increase batch size
    class_mode='categorical'
)

Found 353 images belonging to 7 classes.


In [ ]:
Create a CNN model using the keras API:

In [99]:
# Define the CNN model architecture
model = keras.Sequential([
layers.Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
layers.MaxPooling2D((2, 2)),
layers.Flatten(),
layers.Dense(128, activation='relu'),
layers.Dropout(0.2),
layers.Dense(7, activation='softmax') # 144 classes of animals
])

In [ ]:
Compile the model

Compile the model with a suitable optimizer and loss function:



In [100]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
Step 5: Train the model

Train the model using the fit method:

In [101]:
# Train the model
history = model.fit(
    train_datagen,
    epochs=50,
    validation_data=train_datagen,
    verbose=1
)

Epoch 1/50
45/45 [==============================] - 9s 173ms/step - loss: 3.1074 - accuracy: 0.1785 - val_loss: 1.7923 - val_accuracy: 0.2805
Epoch 2/50
45/45 [==============================] - 8s 170ms/step - loss: 1.7626 - accuracy: 0.2663 - val_loss: 1.5913 - val_accuracy: 0.4023
Epoch 3/50
45/45 [==============================] - 8s 168ms/step - loss: 1.5808 - accuracy: 0.3569 - val_loss: 1.4654 - val_accuracy: 0.4788
Epoch 4/50
45/45 [==============================] - 7s 164ms/step - loss: 1.5041 - accuracy: 0.4674 - val_loss: 1.2197 - val_accuracy: 0.5977
Epoch 5/50
45/45 [==============================] - 8s 170ms/step - loss: 1.4329 - accuracy: 0.4646 - val_loss: 1.2574 - val_accuracy: 0.5581
Epoch 6/50
45/45 [==============================] - 8s 169ms/step - loss: 1.2663 - accuracy: 0.5354 - val_loss: 1.1044 - val_accuracy: 0.6176
Epoch 7/50
45/45 [==============================] - 8s 169ms/step - loss: 1.1920 - accuracy: 0.5779 - val_loss: 1.0686 - val_accuracy: 0.6374
Epoch 

In [121]:
#Export CNN Model
import tensorflow as tf
# Save the Keras model to a file
model.save('animal_classification_model.pb') #or .h5 , it's almost same for deployment on apps but pb is more convertable.


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x78d45019da20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function trace_model_call.<locals>._wrapped_model at 0x78d45019da20>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [125]:
#Testing The Model
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
from google.colab import files
import os

# Load the trained model
model = tf.keras.models.load_model('animal_classification_model.pb')

# Get the list of folders (classes)
folders = os.listdir('/content/gdrive/MyDrive/Sample')

# Upload an image from your PC
uploaded = files.upload()

# Get the uploaded image file
image_file = list(uploaded.keys())[0]

# Open the image file using PIL
image = Image.open(image_file)

# Resize the image to the model's input shape
image = image.resize((128, 128))

# Convert the image to a numpy array
image_array = np.array(image) / 255.0

# Add a batch dimension to the image array
image_array = np.expand_dims(image_array, axis=0)

# Make predictions on the uploaded image
predictions = model.predict(image_array)

# Get the predicted class index
predicted_index = np.argmax(predictions)

# Get the corresponding folder name (class)
predicted_class = folders[predicted_index]

# Print the predicted class
print(f"Predicted class: {predicted_class}")

Saving sand-cat-1024x768.jpg.webp to sand-cat-1024x768.jpg (6).webp
1/1 [==============================] - 0s 61ms/step
Predicted class: sand cat


In [126]:
#Save Indexes for Later Use (json and csv)
import os
import json
import csv

# Get the list of folders (classes)
folders = os.listdir('/content/gdrive/MyDrive/Sample')

# Create a dictionary to map indexes to labels
index_to_label = {i: folder for i, folder in enumerate(folders)}

# Export to JSON file
with open('index_to_label.json', 'w') as f:
    json.dump(index_to_label, f)

# Export to CSV file
with open('index_to_label.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Index', 'Label'])
    for index, label in index_to_label.items():
        writer.writerow([index, label])

In [76]:
#Convert to Tensorflow lite for mobile use

import tensorflow as tf

model = tf.keras.models.load_model('animal_classification_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("animal_classification_model", "wb").write(tflite_model)

32521672